In [ ]:
import sys
import time
import logging
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tf_pose import common
import tf_slim as slim
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh

In [ ]:
def keypoints(human):
    scores = []
    keypoints = str(str(str(human).split('BodyPart:')[1:]).split('-')).split(' score=')
    keypoints_list=[]
    for i in range (len(keypoints)-1): 
        pnt = keypoints[i][-11:-1]
        pnt = tuple(map(float, pnt.split(', ')))
        keypoints_list.append(pnt)
        scores.append(float(str(keypoints[i]).split('score=')[1][0:4]))
   # human=np.array(human)

    
        
    keypts_array = np.array(keypoints_list)
    keypts_array = keypts_array*(image.shape[1],image.shape[0])
    keypts_array = keypts_array.astype(int)
    return(keypts_array, scores)

In [ ]:
model='mobilenet_thin'
resize='480x432'
w, h = model_wh(resize)

In [ ]:
e = TfPoseEstimator(get_graph_path(model), target_size=(w, h))

In [ ]:
video_path = '/Users/andrei-macpro/Documents/Data/excerpt.mp4'

In [ ]:
cap = cv2.VideoCapture(video_path)

In [ ]:
scores = []
if cap.isOpened():
    while True:
        ret_val, image = cap.read()
        frame_no+=1
        humans = e.inference(image,
                             resize_to_default=(w > 0 and h > 0),
                             upsample_size=4.0)
        
        humans_0 = np.array(humans[0])
        keypoints = str(str(str(humans[0]).split('BodyPart:')[1:]).split('-')).split(' score=')
        print((str(humans_0).split('score=')[1][0:4]))
        scores.append(x[1][0:4] for x in str(humans_0).split('score='))
       
        print(scores)
        keypoints_list=[]
        for i in range (len(keypoints)-1): 
            pnt = keypoints[i][-11:-1]
            score = keypoints[i][-10]
            pnt = tuple(map(float, pnt.split(', ')))
            print(type(pnt))
            keypoints_list.append(pnt)
        keypts_array = np.array(keypoints_list)
        print(keypts_array)
        keypts_array = keypts_array*(image.shape[1],image.shape[0])

In [ ]:
scores

In [ ]:
showBG = True
fps_time = 0
if cap.isOpened():
    while True:
        ret_val, image = cap.read()
        humans = e.inference(image,
                             resize_to_default=(w > 0 and h > 0),
                             upsample_size=4.0)
        keypoints = str(str(str(humans[0]).split('BodyPart:')[1:]).split('-')).split(' score=')
        print(keypoints)
        keypoints_list=[]
        for i in range (len(keypoints)-1): 
            pnt = keypoints[i][-11:-1]
            pnt = tuple(map(float, pnt.split(', ')))
            print(pnt)
            keypoints_list.append(pnt)
        keypts_array = np.array(keypoints_list)
        print(keypts_array)
        keypts_array = keypts_array*(image.shape[1],image.shape[0])
        print(image.shape)
        keypts_array = keypts_array.astype(int)
        print(f"keypoints for person 1 {keypts_array}")
        minimum = np.amin(keypts_array, axis=0)
        maximum = np.amax(keypts_array, axis=0)
        #rect = cv2.rectangle(image,(minimum[0]-20, minimum[1]-20),(maximum[0]+20,maximum[1]+20),(0,255,0),2)
        #cv2.boundingRect(minimum[0]-20:minimum[1]-20,maximum[0]+20:maximum[1]+20)
        #x,y,w,h = cv2.boundingRect((minimum[0]-20, minimum[1]-20,maximum[0]+20,maximum[1]+20))
        #roi=image[y:y+h,x:x+w]
        #https://stackoverflow.com/questions/54054865/given-pixel-label-draw-a-bounding-box-in-python
        #cv2.imshow('rect.png', rect)
        #image = np.zeros(image.shape)
        #cv2.imshow('image', image)
        rect = image[minimum[1]:maximum[1],minimum[0]:maximum[0]]
        cv2.imwrite('rect.png', rect)
        image = TfPoseEstimator.draw_humans(image, humans, imgcopy=True)
       # cv2.putText(image, "FPS: %f" % (1.0 / (time.time() - fps_time)), (10, 10),
        #            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
       # cv2.imshow('tf-pose-estimation result', image)
        #fps_time = time.time()
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [ ]:
scores

In [ ]:
(0.44, 0.12) *(image.shape[1],image.shape[0])

In [ ]:
from operator import mul
res = tuple(map(mul, (0.44, 0.12), (image.shape[1],image.shape[0])))

In [ ]:
tuple(map(float, res))

In [ ]:
type(res)

In [ ]:
res2 = np.array(res)


In [ ]:
res2.astype(int)

In [ ]:
def keypoints(human):
    scores = []
    keypoints = str(str(str(human).split('BodyPart:')[1:]).split('-')).split(' score=')
    keypoints_list=[]
    for i in range (len(keypoints)-1): 
        pnt = keypoints[i][-11:-1]
        pnt = tuple(map(float, pnt.split(', ')))
        keypoints_list.append(pnt)
    human=np.array(human)
    for i in str(human).split(" score="):
        if i[0:4]=="Body":
            continue
        else:
            scores.append(float(i[0:4]))
    scores=np.array(scores)
    keypts_array = np.array(keypoints_list)
    keypts_array = keypts_array*(image.shape[1],image.shape[0])
    keypts_array = keypts_array.astype(int)
    return(keypts_array, scores)

In [ ]:
cap = cv2.VideoCapture(video_path)
while(cap.isOpened()):
    ret_val, image = cap.read()
    if ret_val == False:
        break
    humans = e.inference(image,
                         resize_to_default=(w > 0 and h > 0),
                         upsample_size=4.0)
    
    keypoints = str(str(str(humans[0]).split('BodyPart:')[1:]).split('-')).split(' score=')
    keypoints_list=[]
    humans = np.array(humans[0])
    print(humans)
    for i in str(humans).split(" score="):
        if i[0:4]=="Body":
            continue
        else:
            print(i[0:4])

In [ ]:
cap = cv2.VideoCapture(video_path)
human_no = []
frame_no = 0
failed_frames = []
human_1 = []
human_2 = []
human_3 =[]
probs_1 = []
while(cap.isOpened()):
    ret_val, image = cap.read()
    if ret_val == False:
        break
    frame_no+=1
    humans = e.inference(image,
                         resize_to_default=(w > 0 and h > 0),
                         upsample_size=4.0)
    human_no.append(len(humans))
    
    if len(humans)>2:
        keypoints_3 = keypoints(humans[2])
        keypoints_2 = keypoints(humans[1])
        keypoints_1 = keypoints(humans[0])
        failed_frames.append('success')
    
    elif len(humans)>1:
        keypoints_2 = keypoints(humans[1])
        keypoints_1 = keypoints(humans[0])
        failed_frames.append('success')
    elif len(humans) == 1:
        keypoints_1 = keypoints(humans[0])
        failed_frames.append('success')
    else:
        failed_frames.append(frame_no)
    human_1.append({frame_no:keypoints_1})
    human_2.append({frame_no:keypoints_2})
    human_3.append({frame_no:keypoints_3})
 

    
 
cap.release()
human_1 = np.array(human_1)
human_2 = np.array(human_2)
human_3 = np.array(human_3)
human_no = np.array(human_no)
failed_frames = np.array(failed_frames) 

In [ ]:
failed_frames

In [ ]:
human_1[0]

In [ ]:
for i in range(18):
    print(i)

In [ ]:

cap = cv2.VideoCapture(video_path)
while(cap.isOpened()):
    ret_val, image = cap.read()
    if ret_val == False:
        break
    humans = e.inference(image,
                         resize_to_default=(w > 0 and h > 0),
                         upsample_size=4.0)
    human_0 = humans[0]

In [ ]:
human_0

In [ ]:
(str(human_0).split("-"))

In [ ]:
showBG = True
fps_time = 0
if cap.isOpened():
    while True:
        ret_val, image = cap.read()
        humans = e.inference(image,
                             resize_to_default=(w > 0 and h > 0),
                             upsample_size=4.0)
        keypoints = str(str(str(humans[0]).split('BodyPart:')[1:]).split('-')).split(' score=')
        print(keypoints)
        keypoints_list=[]
        for i in range (len(keypoints)-1): 
            pnt = keypoints[i][-11:-1]
            pnt = tuple(map(float, pnt.split(', ')))
            print(pnt)
            keypoints_list.append(pnt)
        keypts_array = np.array(keypoints_list)
        print(keypts_array)
        keypts_array = keypts_array*(image.shape[1],image.shape[0])
        print(image.shape)
        keypts_array = keypts_array.astype(int)
        print(f"keypoints for person 1 {keypts_array}")
        minimum = np.amin(keypts_array, axis=0)
        maximum = np.amax(keypts_array, axis=0)
        #rect = cv2.rectangle(image,(minimum[0]-20, minimum[1]-20),(maximum[0]+20,maximum[1]+20),(0,255,0),2)
        #cv2.boundingRect(minimum[0]-20:minimum[1]-20,maximum[0]+20:maximum[1]+20)
        #x,y,w,h = cv2.boundingRect((minimum[0]-20, minimum[1]-20,maximum[0]+20,maximum[1]+20))
        #roi=image[y:y+h,x:x+w]
        #https://stackoverflow.com/questions/54054865/given-pixel-label-draw-a-bounding-box-in-python
        #cv2.imshow('rect.png', rect)
        #image = np.zeros(image.shape)
        #cv2.imshow('image', image)
        rect = image[minimum[1]:maximum[1],minimum[0]:maximum[0]]
        cv2.imwrite('rect.png', rect)
        image = TfPoseEstimator.draw_humans(image, humans, imgcopy=True)
       # cv2.putText(image, "FPS: %f" % (1.0 / (time.time() - fps_time)), (10, 10),
        #            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
       # cv2.imshow('tf-pose-estimation result', image)
        #fps_time = time.time()
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [ ]:


subscriptable = str(human_0).split("BodyPart:")[1:]


In [ ]:
kpts = []
coords_x = []
coords_y = []
probs = []
all_points = [x for x in range(18)]
big_dic = []
subscriptable = str(human_0).split("BodyPart:")[1:]
for x in subscriptable:
    kpts.append(int((x.split('-')[0])))
    coords_x.append(float((str(x.split('-')[1]).split(" score=")[0][1:5]))*image.shape[1])
    coords_y.append(float((str(x.split('-')[1]).split(" score=")[0][6:11]))*image.shape[0])
    probs.append(float(str(x.split('-')[1]).split(" score=")[1]))
for x,y,cox, coy, prob in zip(all_points,kpts, coords_x, coords_y, probs):
    if x not in kpts:
        big_dic.append({x:['no_x','no_y','no_prob']})
    else:
        big_dic.append({y:[round(cox,2),round(coy,2), prob]})

#return(np.array(kpts), np.array(coords_x), 

In [ ]:
huh = str(human_0).replace('BodyPart:', "").replace("score=", "")
huh

In [ ]:
big_dic

In [ ]:
keypoint = []
for i in range(len(keyp_coord)):
    keypoint.append(int(keyp_coord[i][2]))

In [ ]:
str(human_0).find('score=')

In [ ]:
import json
json.loads(human_0)